# Chapter 4: Backtesting, Evaluation and Portfolio Analysis
# 第四章：回测、评估与组合分析

**Author**: Microsoft Qlib Team  
**License**: MIT License  
**Last Updated**: 2025-01-09

---

## 📚 Table of Contents / 目录

1. [Evaluation Framework Overview / 评估框架概览](#evaluation-framework)
2. [Backtesting Infrastructure / 回测基础设施](#backtesting)
3. [Trading Strategies / 交易策略](#strategies)
   - 3.1 [Top-K Strategy](#topk-strategy)
   - 3.2 [Signal-based Strategies](#signal-strategies)
   - 3.3 [Custom Strategies](#custom-strategies)
4. [Execution and Order Management / 执行与订单管理](#execution)
5. [Portfolio Analysis / 组合分析](#portfolio-analysis)
6. [Risk Metrics / 风险指标](#risk-metrics)
7. [Performance Attribution / 业绩归因](#attribution)
8. [Signal Analysis / 信号分析](#signal-analysis)
9. [Transaction Cost Analysis / 交易成本分析](#tca)
10. [Visualization and Reporting / 可视化与报告](#visualization)
11. [Advanced Evaluation Techniques / 高级评估技术](#advanced)
12. [Production Monitoring / 生产监控](#monitoring)

## Setup and Imports / 设置和导入

In [ ]:
# Essential imports / 必要导入
import qlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import pickle
import warnings
from typing import Dict, List, Tuple, Optional
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')

# Qlib specific imports / Qlib特定导入
from qlib.data import D
from qlib.config import C
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord, SigAnaRecord
from qlib.utils import init_instance_by_config

# Backtesting imports / 回测导入
from qlib.backtest import backtest
from qlib.backtest.executor import SimulatorExecutor
from qlib.backtest.exchange import Exchange
from qlib.backtest.account import Account

# Strategy imports / 策略导入
from qlib.contrib.strategy.signal_strategy import TopkDropoutStrategy
from qlib.contrib.evaluate import risk_analysis
from qlib.contrib.report import analysis_position, analysis_model

# Visualization settings / 可视化设置
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
sns.set_palette("husl")

print("✅ Libraries imported successfully")

In [ ]:
# Initialize Qlib and load configuration / 初始化Qlib并加载配置
qlib.init()

# Load experiment configuration / 加载实验配置
try:
    with open('experiment_config.json', 'r') as f:
        exp_config = json.load(f)
    print("✅ Experiment configuration loaded")
except FileNotFoundError:
    exp_config = {
        "market": "csi300",
        "benchmark": "SH000300",
        "exp_name": "evaluation_exp",
        "train_start": "2008-01-01",
        "train_end": "2014-12-31",
        "valid_start": "2015-01-01",
        "valid_end": "2016-12-31",
        "test_start": "2017-01-01",
        "test_end": "2020-08-01"
    }
    print("⚠️ Using default configuration")

# Load saved model and predictions / 加载保存的模型和预测
try:
    model_dir = Path('saved_models')
    with open(model_dir / 'predictions.pkl', 'rb') as f:
        predictions = pickle.load(f)
    print("✅ Predictions loaded")
except FileNotFoundError:
    print("⚠️ No saved predictions found. Generating sample predictions...")
    # Generate sample predictions for demonstration
    dates = pd.date_range(exp_config['test_start'], exp_config['test_end'], freq='D')
    instruments = D.instruments(exp_config['market'])[:50]
    index = pd.MultiIndex.from_product([dates, instruments], names=['datetime', 'instrument'])
    predictions = pd.Series(np.random.randn(len(index)) * 0.01, index=index, name='score')

print(f"\nExperiment: {exp_config['exp_name']}")
print(f"Test period: {exp_config['test_start']} to {exp_config['test_end']}")
print(f"Predictions shape: {predictions.shape}")

## 1. Evaluation Framework Overview / 评估框架概览 <a id='evaluation-framework'></a>

```
┌──────────────────────────────────────────────────────────────────────┐
│                    Qlib Evaluation Framework                         │
├──────────────────────────────────────────────────────────────────────┤
│                                                                       │
│     Model Predictions                                                │
│            ↓                                                         │
│   ┌─────────────────┐                                               │
│   │ Signal Analysis │ ← IC, Rank IC, Auto-correlation              │
│   └─────────────────┘                                               │
│            ↓                                                         │
│   ┌─────────────────┐                                               │
│   │    Strategy     │ ← Top-K, Dropout, Signal-based              │
│   └─────────────────┘                                               │
│            ↓                                                         │
│   ┌─────────────────┐                                               │
│   │    Executor     │ ← Order generation, Position management      │
│   └─────────────────┘                                               │
│            ↓                                                         │
│   ┌─────────────────┐                                               │
│   │    Exchange     │ ← Trade execution, Cost calculation         │
│   └─────────────────┘                                               │
│            ↓                                                         │
│   ┌─────────────────────────────────────┐                         │
│   │     Performance Analysis            │                         │
│   │  • Returns & Risk Metrics           │                         │
│   │  • Portfolio Analytics              │                         │
│   │  • Attribution Analysis             │                         │
│   └─────────────────────────────────────┘                         │
└──────────────────────────────────────────────────────────────────────┘
```

## 2. Backtesting Infrastructure / 回测基础设施 <a id='backtesting'></a>

In [ ]:
# Basic backtesting configuration / 基础回测配置

backtest_config = {
    "start_time": exp_config['test_start'],
    "end_time": exp_config['test_end'],
    "account": 10000000,  # Initial capital: 10M / 初始资金：1000万
    "benchmark": exp_config['benchmark'],
    "exchange_kwargs": {
        "freq": "day",
        "limit_threshold": 0.095,  # Daily limit 9.5% / 涨跌停限制9.5%
        "deal_price": "close",      # Trade at close price / 以收盘价成交
        "open_cost": 0.0005,        # Buy cost 0.05% / 买入成本0.05%
        "close_cost": 0.0015,       # Sell cost 0.15% / 卖出成本0.15%
        "min_cost": 5,              # Minimum cost 5 yuan / 最低手续费5元
    },
}

print("Backtest Configuration / 回测配置:")
print("="*50)
for key, value in backtest_config.items():
    if key != "exchange_kwargs":
        print(f"{key:15}: {value}")

print("\nExchange Settings / 交易所设置:")
for key, value in backtest_config["exchange_kwargs"].items():
    print(f"  {key:15}: {value}")

In [ ]:
# Create exchange and account / 创建交易所和账户

from qlib.backtest import Exchange, Account

# Initialize exchange / 初始化交易所
exchange = Exchange(
    trade_dates=D.calendar(start_time=backtest_config['start_time'], 
                          end_time=backtest_config['end_time']),
    **backtest_config['exchange_kwargs']
)

# Initialize account / 初始化账户
account = Account(init_cash=backtest_config['account'])

print(f"Exchange initialized with {len(exchange.trade_dates)} trading days")
print(f"Account initialized with ${backtest_config['account']:,.0f} capital")

# Display trading calendar sample / 显示交易日历样例
print(f"\nFirst 5 trading days: {exchange.trade_dates[:5].tolist()}")
print(f"Last 5 trading days: {exchange.trade_dates[-5:].tolist()}")

## 3. Trading Strategies / 交易策略 <a id='strategies'></a>

### 3.1 Top-K Strategy <a id='topk-strategy'></a>

In [ ]:
# Top-K Dropout Strategy Implementation / Top-K Dropout策略实现

from qlib.contrib.strategy.signal_strategy import TopkDropoutStrategy

# Strategy configuration / 策略配置
topk_strategy_config = {
    "signal": predictions,           # Model predictions / 模型预测
    "topk": 50,                     # Select top 50 stocks / 选择前50只股票
    "n_drop": 5,                    # Drop worst 5 each period / 每期淘汰最差的5只
    "method": "top",                # Selection method / 选择方法
    "risk_degree": 0.95,            # Risk control parameter / 风险控制参数
    "hold_thresh": 0,               # Holding threshold / 持有阈值
    "only_tradable": True,          # Only trade tradable stocks / 仅交易可交易股票
}

# Create strategy instance / 创建策略实例
topk_strategy = TopkDropoutStrategy(**topk_strategy_config)

print("Top-K Dropout Strategy Configuration:")
print(f"  Top K: {topk_strategy_config['topk']}")
print(f"  Dropout: {topk_strategy_config['n_drop']}")
print(f"  Risk degree: {topk_strategy_config['risk_degree']}")
print(f"  Method: {topk_strategy_config['method']}")

In [ ]:
# Run backtest with Top-K strategy / 运行Top-K策略回测

from qlib.backtest import backtest
from qlib.backtest.executor import SimulatorExecutor

# Create executor / 创建执行器
executor = SimulatorExecutor(
    time_per_step="day",
    generate_portfolio_metrics=True
)

# Run backtest / 运行回测
print("Running backtest...")
portfolio_metrics, indicator_dict = backtest(
    strategy=topk_strategy,
    executor=executor,
    backtest_config=backtest_config,
    account=backtest_config["account"],
    benchmark=backtest_config["benchmark"],
    exchange_kwargs=backtest_config["exchange_kwargs"],
    return_order=True
)

print("✅ Backtest completed")
print(f"\nPortfolio metrics keys: {list(portfolio_metrics.keys())}")
print(f"Indicator dict keys: {list(indicator_dict.keys())}")

### 3.2 Signal-based Strategies <a id='signal-strategies'></a>

In [ ]:
# Custom signal-based strategy / 自定义信号策略

from qlib.contrib.strategy.signal_strategy import SignalStrategy
from qlib.backtest.decision import BaseTradeDecision, Order

class MomentumStrategy(SignalStrategy):
    """Momentum-based trading strategy / 基于动量的交易策略"""
    
    def __init__(self, signal, momentum_window=20, rebalance_freq=5, **kwargs):
        super().__init__(signal=signal, **kwargs)
        self.momentum_window = momentum_window
        self.rebalance_freq = rebalance_freq
        self.trade_count = 0
    
    def generate_trade_decision(self, execute_result=None):
        """Generate trading decisions / 生成交易决策"""
        
        # Get current date / 获取当前日期
        current_date = self.trade_calendar.get_trade_date()
        
        # Check if it's rebalance day / 检查是否为再平衡日
        self.trade_count += 1
        if self.trade_count % self.rebalance_freq != 0:
            return BaseTradeDecision([], current_date)
        
        # Get current signal / 获取当前信号
        current_signal = self.signal.loc[current_date] if current_date in self.signal.index else pd.Series()
        
        if current_signal.empty:
            return BaseTradeDecision([], current_date)
        
        # Select top momentum stocks / 选择动量最强的股票
        top_stocks = current_signal.nlargest(30)
        
        # Generate orders / 生成订单
        orders = []
        position_per_stock = 1.0 / len(top_stocks) if len(top_stocks) > 0 else 0
        
        for stock, score in top_stocks.items():
            if score > 0:  # Only buy positive momentum / 只买入正动量
                order = Order(
                    stock_id=stock,
                    amount=position_per_stock,
                    direction=Order.BUY,
                    start_time=current_date,
                    end_time=current_date
                )
                orders.append(order)
        
        return BaseTradeDecision(orders, current_date)

# Create momentum strategy / 创建动量策略
momentum_strategy = MomentumStrategy(
    signal=predictions,
    momentum_window=20,
    rebalance_freq=5
)

print("Momentum Strategy created")
print(f"  Momentum window: 20 days")
print(f"  Rebalance frequency: Every 5 days")

### 3.3 Custom Strategies <a id='custom-strategies'></a>

In [ ]:
# Advanced custom strategy with risk management / 带风险管理的高级自定义策略

class RiskManagedStrategy(SignalStrategy):
    """Strategy with position sizing and risk management
    带仓位管理和风险控制的策略
    """
    
    def __init__(self, signal, max_position=0.1, stop_loss=0.05, 
                 volatility_target=0.15, **kwargs):
        super().__init__(signal=signal, **kwargs)
        self.max_position = max_position      # Maximum position per stock / 每只股票最大仓位
        self.stop_loss = stop_loss            # Stop loss threshold / 止损阈值
        self.volatility_target = volatility_target  # Target portfolio volatility / 目标组合波动率
        self.entry_prices = {}                # Track entry prices / 跟踪入场价格
        
    def calculate_position_size(self, stock, signal_strength, current_volatility):
        """Calculate position size based on signal and volatility
        基于信号和波动率计算仓位大小
        """
        
        # Base position from signal strength / 基于信号强度的基础仓位
        base_position = abs(signal_strength) * self.max_position
        
        # Adjust for volatility / 根据波动率调整
        if current_volatility > 0:
            volatility_scalar = self.volatility_target / current_volatility
            volatility_scalar = min(volatility_scalar, 2.0)  # Cap at 2x
            volatility_scalar = max(volatility_scalar, 0.5)  # Floor at 0.5x
        else:
            volatility_scalar = 1.0
        
        # Final position size / 最终仓位大小
        position_size = base_position * volatility_scalar
        position_size = min(position_size, self.max_position)
        
        return position_size
    
    def check_stop_loss(self, stock, current_price):
        """Check if stop loss is triggered / 检查是否触发止损"""
        
        if stock not in self.entry_prices:
            return False
        
        entry_price = self.entry_prices[stock]
        loss = (entry_price - current_price) / entry_price
        
        return loss > self.stop_loss
    
    def generate_trade_decision(self, execute_result=None):
        """Generate risk-managed trading decisions
        生成风险管理的交易决策
        """
        
        current_date = self.trade_calendar.get_trade_date()
        current_signal = self.signal.loc[current_date] if current_date in self.signal.index else pd.Series()
        
        if current_signal.empty:
            return BaseTradeDecision([], current_date)
        
        orders = []
        
        # Get current prices and calculate volatility / 获取当前价格并计算波动率
        # Note: Simplified for demonstration / 注意：为演示而简化
        current_volatility = 0.20  # Placeholder / 占位符
        
        for stock, signal_value in current_signal.items():
            # Calculate position size / 计算仓位大小
            position_size = self.calculate_position_size(
                stock, signal_value, current_volatility
            )
            
            # Generate order if position size is significant / 如果仓位显著则生成订单
            if position_size > 0.01:
                direction = Order.BUY if signal_value > 0 else Order.SELL
                order = Order(
                    stock_id=stock,
                    amount=position_size,
                    direction=direction,
                    start_time=current_date,
                    end_time=current_date
                )
                orders.append(order)
                
                # Track entry price / 跟踪入场价格
                if direction == Order.BUY:
                    self.entry_prices[stock] = 100  # Placeholder price
        
        return BaseTradeDecision(orders, current_date)

# Create risk-managed strategy / 创建风险管理策略
risk_strategy = RiskManagedStrategy(
    signal=predictions,
    max_position=0.1,
    stop_loss=0.05,
    volatility_target=0.15
)

print("Risk-Managed Strategy created")
print(f"  Max position per stock: 10%")
print(f"  Stop loss: 5%")
print(f"  Volatility target: 15%")

## 4. Execution and Order Management / 执行与订单管理 <a id='execution'></a>

In [ ]:
# Custom executor with advanced features / 带高级功能的自定义执行器

from qlib.backtest.executor import BaseExecutor
from qlib.backtest.exchange import Exchange
from qlib.backtest.account import Account

class AdvancedExecutor(BaseExecutor):
    """Advanced executor with slippage and market impact modeling
    带滑点和市场冲击建模的高级执行器
    """
    
    def __init__(self, time_per_step="day", slippage_rate=0.001, 
                 market_impact_model="linear", **kwargs):
        super().__init__(time_per_step=time_per_step, **kwargs)
        self.slippage_rate = slippage_rate
        self.market_impact_model = market_impact_model
        self.execution_history = []
    
    def calculate_slippage(self, order_amount, avg_volume):
        """Calculate slippage based on order size and volume
        基于订单大小和成交量计算滑点
        """
        
        # Basic slippage model / 基础滑点模型
        base_slippage = self.slippage_rate
        
        # Adjust for order size relative to volume / 根据订单相对成交量调整
        if avg_volume > 0:
            size_impact = order_amount / avg_volume
            size_impact = min(size_impact, 0.1)  # Cap at 10% of volume
        else:
            size_impact = 0.01
        
        total_slippage = base_slippage * (1 + size_impact * 10)
        
        return total_slippage
    
    def calculate_market_impact(self, order_amount, market_cap):
        """Calculate market impact of large orders
        计算大额订单的市场冲击
        """
        
        if self.market_impact_model == "linear":
            # Linear impact model / 线性冲击模型
            impact = order_amount / market_cap * 0.1
        elif self.market_impact_model == "sqrt":
            # Square root impact model / 平方根冲击模型
            impact = np.sqrt(order_amount / market_cap) * 0.05
        else:
            impact = 0
        
        return min(impact, 0.02)  # Cap at 2% impact
    
    def execute_orders(self, orders, current_date):
        """Execute orders with realistic constraints
        以现实约束执行订单
        """
        
        executed_orders = []
        
        for order in orders:
            # Simulate execution with slippage / 模拟带滑点的执行
            avg_volume = 1000000  # Placeholder / 占位符
            market_cap = 1000000000  # Placeholder / 占位符
            
            slippage = self.calculate_slippage(order.amount, avg_volume)
            market_impact = self.calculate_market_impact(order.amount, market_cap)
            
            # Adjust execution price / 调整执行价格
            execution_cost = slippage + market_impact
            
            # Record execution / 记录执行
            self.execution_history.append({
                'date': current_date,
                'stock': order.stock_id,
                'amount': order.amount,
                'slippage': slippage,
                'market_impact': market_impact,
                'total_cost': execution_cost
            })
            
            executed_orders.append(order)
        
        return executed_orders

# Create advanced executor / 创建高级执行器
advanced_executor = AdvancedExecutor(
    time_per_step="day",
    slippage_rate=0.001,
    market_impact_model="linear"
)

print("Advanced Executor created")
print(f"  Slippage rate: 0.1%")
print(f"  Market impact model: linear")

## 5. Portfolio Analysis / 组合分析 <a id='portfolio-analysis'></a>

In [ ]:
# Comprehensive portfolio analysis / 综合组合分析

def analyze_portfolio(portfolio_metrics, benchmark_returns=None):
    """Comprehensive portfolio analysis
    综合组合分析
    """
    
    # Extract key metrics / 提取关键指标
    if 'return' in portfolio_metrics:
        returns = portfolio_metrics['return']
    else:
        # Generate sample returns for demonstration
        dates = pd.date_range(exp_config['test_start'], exp_config['test_end'], freq='D')
        returns = pd.Series(np.random.randn(len(dates)) * 0.01, index=dates)
    
    # Calculate cumulative returns / 计算累积收益
    cumulative_returns = (1 + returns).cumprod()
    
    # Performance metrics / 性能指标
    metrics = {
        'Total Return': cumulative_returns.iloc[-1] - 1,
        'Annual Return': (cumulative_returns.iloc[-1] ** (252/len(returns))) - 1,
        'Volatility': returns.std() * np.sqrt(252),
        'Sharpe Ratio': (returns.mean() / returns.std()) * np.sqrt(252) if returns.std() > 0 else 0,
        'Max Drawdown': calculate_max_drawdown(cumulative_returns),
        'Calmar Ratio': 0,  # Will calculate below
        'Win Rate': (returns > 0).mean(),
        'Best Day': returns.max(),
        'Worst Day': returns.min(),
        'Skewness': returns.skew(),
        'Kurtosis': returns.kurtosis()
    }
    
    # Calculate Calmar ratio / 计算Calmar比率
    if metrics['Max Drawdown'] != 0:
        metrics['Calmar Ratio'] = metrics['Annual Return'] / abs(metrics['Max Drawdown'])
    
    return metrics, returns, cumulative_returns

def calculate_max_drawdown(cumulative_returns):
    """Calculate maximum drawdown / 计算最大回撤"""
    running_max = cumulative_returns.expanding().max()
    drawdown = (cumulative_returns - running_max) / running_max
    return drawdown.min()

# Analyze portfolio / 分析组合
metrics, returns, cumulative_returns = analyze_portfolio(portfolio_metrics)

# Display metrics / 显示指标
print("Portfolio Performance Metrics / 组合性能指标:")
print("="*50)
for metric, value in metrics.items():
    if isinstance(value, float):
        if 'Return' in metric or 'Rate' in metric or 'Volatility' in metric:
            print(f"{metric:20}: {value:>10.2%}")
        else:
            print(f"{metric:20}: {value:>10.4f}")
    else:
        print(f"{metric:20}: {value:>10}")

In [ ]:
# Portfolio visualization / 组合可视化

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Portfolio Analysis Dashboard / 组合分析仪表板', fontsize=16)

# 1. Cumulative returns / 累积收益
axes[0, 0].plot(cumulative_returns.index, cumulative_returns.values, linewidth=2)
axes[0, 0].set_title('Cumulative Returns / 累积收益')
axes[0, 0].set_xlabel('Date / 日期')
axes[0, 0].set_ylabel('Cumulative Return / 累积收益')
axes[0, 0].grid(True, alpha=0.3)

# 2. Daily returns distribution / 日收益分布
axes[0, 1].hist(returns, bins=50, alpha=0.7, edgecolor='black')
axes[0, 1].axvline(x=0, color='red', linestyle='--', alpha=0.5)
axes[0, 1].set_title('Returns Distribution / 收益分布')
axes[0, 1].set_xlabel('Daily Return / 日收益')
axes[0, 1].set_ylabel('Frequency / 频率')
axes[0, 1].grid(True, alpha=0.3)

# 3. Rolling volatility / 滚动波动率
rolling_vol = returns.rolling(window=60).std() * np.sqrt(252)
axes[0, 2].plot(rolling_vol.index, rolling_vol.values, linewidth=2, color='orange')
axes[0, 2].set_title('60-Day Rolling Volatility / 60日滚动波动率')
axes[0, 2].set_xlabel('Date / 日期')
axes[0, 2].set_ylabel('Annualized Volatility / 年化波动率')
axes[0, 2].grid(True, alpha=0.3)

# 4. Drawdown / 回撤
running_max = cumulative_returns.expanding().max()
drawdown = (cumulative_returns - running_max) / running_max
axes[1, 0].fill_between(drawdown.index, drawdown.values, 0, alpha=0.7, color='red')
axes[1, 0].set_title('Drawdown / 回撤')
axes[1, 0].set_xlabel('Date / 日期')
axes[1, 0].set_ylabel('Drawdown / 回撤')
axes[1, 0].grid(True, alpha=0.3)

# 5. Rolling Sharpe ratio / 滚动夏普比率
rolling_sharpe = (returns.rolling(window=60).mean() / returns.rolling(window=60).std()) * np.sqrt(252)
axes[1, 1].plot(rolling_sharpe.index, rolling_sharpe.values, linewidth=2, color='green')
axes[1, 1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1, 1].set_title('60-Day Rolling Sharpe Ratio / 60日滚动夏普比率')
axes[1, 1].set_xlabel('Date / 日期')
axes[1, 1].set_ylabel('Sharpe Ratio / 夏普比率')
axes[1, 1].grid(True, alpha=0.3)

# 6. Q-Q plot / Q-Q图
from scipy import stats
stats.probplot(returns, dist="norm", plot=axes[1, 2])
axes[1, 2].set_title('Q-Q Plot / Q-Q图')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Risk Metrics / 风险指标 <a id='risk-metrics'></a>

In [ ]:
# Comprehensive risk analysis / 综合风险分析

def calculate_risk_metrics(returns, confidence_level=0.95):
    """Calculate comprehensive risk metrics
    计算综合风险指标
    """
    
    # Basic statistics / 基础统计
    risk_metrics = {
        'Volatility (Annual)': returns.std() * np.sqrt(252),
        'Downside Volatility': returns[returns < 0].std() * np.sqrt(252),
        'Upside Volatility': returns[returns > 0].std() * np.sqrt(252),
    }
    
    # Value at Risk (VaR) / 风险价值
    var_percentile = (1 - confidence_level) * 100
    risk_metrics['VaR (95%)'] = np.percentile(returns, var_percentile)
    risk_metrics['VaR (99%)'] = np.percentile(returns, 1)
    
    # Conditional VaR (CVaR) / 条件风险价值
    var_threshold = risk_metrics['VaR (95%)']
    risk_metrics['CVaR (95%)'] = returns[returns <= var_threshold].mean()
    
    # Maximum consecutive losses / 最大连续亏损
    consecutive_losses = 0
    max_consecutive_losses = 0
    for ret in returns:
        if ret < 0:
            consecutive_losses += 1
            max_consecutive_losses = max(max_consecutive_losses, consecutive_losses)
        else:
            consecutive_losses = 0
    risk_metrics['Max Consecutive Losses'] = max_consecutive_losses
    
    # Sortino ratio / 索提诺比率
    downside_returns = returns[returns < 0]
    if len(downside_returns) > 0:
        downside_std = downside_returns.std()
        if downside_std > 0:
            risk_metrics['Sortino Ratio'] = (returns.mean() / downside_std) * np.sqrt(252)
        else:
            risk_metrics['Sortino Ratio'] = np.inf
    else:
        risk_metrics['Sortino Ratio'] = np.inf
    
    # Omega ratio / 欧米伽比率
    threshold = 0
    gains = returns[returns > threshold] - threshold
    losses = threshold - returns[returns <= threshold]
    if losses.sum() > 0:
        risk_metrics['Omega Ratio'] = gains.sum() / losses.sum()
    else:
        risk_metrics['Omega Ratio'] = np.inf
    
    # Tail ratio / 尾部比率
    right_tail = np.percentile(returns, 95)
    left_tail = abs(np.percentile(returns, 5))
    if left_tail > 0:
        risk_metrics['Tail Ratio'] = right_tail / left_tail
    else:
        risk_metrics['Tail Ratio'] = np.inf
    
    return risk_metrics

# Calculate risk metrics / 计算风险指标
risk_metrics = calculate_risk_metrics(returns)

print("Risk Metrics / 风险指标:")
print("="*50)
for metric, value in risk_metrics.items():
    if isinstance(value, float):
        if value == np.inf:
            print(f"{metric:30}: {'Infinity':>15}")
        elif 'Ratio' in metric:
            print(f"{metric:30}: {value:>15.4f}")
        else:
            print(f"{metric:30}: {value:>15.4%}")
    else:
        print(f"{metric:30}: {value:>15}")

In [ ]:
# Risk decomposition / 风险分解

def risk_decomposition(returns, market_returns=None):
    """Decompose portfolio risk into systematic and idiosyncratic components
    将组合风险分解为系统性和特质性成分
    """
    
    # Generate sample market returns if not provided / 如果未提供则生成样本市场收益
    if market_returns is None:
        market_returns = returns + np.random.randn(len(returns)) * 0.005
    
    # Ensure alignment / 确保对齐
    aligned = pd.DataFrame({
        'portfolio': returns,
        'market': market_returns
    }).dropna()
    
    # Calculate beta / 计算贝塔
    covariance = aligned.cov()
    beta = covariance.loc['portfolio', 'market'] / covariance.loc['market', 'market']
    
    # Systematic and idiosyncratic returns / 系统性和特质性收益
    systematic_returns = beta * aligned['market']
    idiosyncratic_returns = aligned['portfolio'] - systematic_returns
    
    # Risk decomposition / 风险分解
    total_variance = aligned['portfolio'].var()
    systematic_variance = systematic_returns.var()
    idiosyncratic_variance = idiosyncratic_returns.var()
    
    decomposition = {
        'Beta': beta,
        'Total Risk (Annual)': np.sqrt(total_variance * 252),
        'Systematic Risk (Annual)': np.sqrt(systematic_variance * 252),
        'Idiosyncratic Risk (Annual)': np.sqrt(idiosyncratic_variance * 252),
        'Systematic Risk %': systematic_variance / total_variance,
        'Idiosyncratic Risk %': idiosyncratic_variance / total_variance,
        'R-squared': (systematic_variance / total_variance) if total_variance > 0 else 0
    }
    
    return decomposition, systematic_returns, idiosyncratic_returns

# Perform risk decomposition / 执行风险分解
decomposition, sys_returns, idio_returns = risk_decomposition(returns)

print("Risk Decomposition / 风险分解:")
print("="*50)
for metric, value in decomposition.items():
    if '%' in metric or 'Risk' in metric:
        print(f"{metric:30}: {value:>15.2%}")
    else:
        print(f"{metric:30}: {value:>15.4f}")

# Visualize risk decomposition / 可视化风险分解
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Risk contribution pie chart / 风险贡献饼图
risk_contrib = [decomposition['Systematic Risk %'], decomposition['Idiosyncratic Risk %']]
axes[0].pie(risk_contrib, labels=['Systematic', 'Idiosyncratic'], 
           autopct='%1.1f%%', startangle=90)
axes[0].set_title('Risk Decomposition / 风险分解')

# Time series of risk components / 风险成分时间序列
axes[1].plot(sys_returns.index, sys_returns.cumsum(), label='Systematic', linewidth=2)
axes[1].plot(idio_returns.index, idio_returns.cumsum(), label='Idiosyncratic', linewidth=2)
axes[1].set_title('Cumulative Risk Components / 累积风险成分')
axes[1].set_xlabel('Date / 日期')
axes[1].set_ylabel('Cumulative Return / 累积收益')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Performance Attribution / 业绩归因 <a id='attribution'></a>

In [ ]:
# Performance attribution analysis / 业绩归因分析

class PerformanceAttribution:
    """Performance attribution framework / 业绩归因框架"""
    
    def __init__(self, portfolio_returns, benchmark_returns, positions=None):
        self.portfolio_returns = portfolio_returns
        self.benchmark_returns = benchmark_returns
        self.positions = positions
    
    def brinson_attribution(self):
        """Brinson attribution model / Brinson归因模型
        Decomposes excess returns into allocation and selection effects
        将超额收益分解为配置效应和选择效应
        """
        
        # Simplified Brinson attribution / 简化的Brinson归因
        # In practice, this would use sector/asset class data
        
        # Total effect / 总效应
        total_effect = self.portfolio_returns.mean() - self.benchmark_returns.mean()
        
        # Allocation effect (simplified) / 配置效应（简化）
        # This would normally compare sector weights
        allocation_effect = total_effect * 0.4  # Placeholder
        
        # Selection effect (simplified) / 选择效应（简化）
        # This would normally compare stock selection within sectors
        selection_effect = total_effect * 0.6  # Placeholder
        
        # Interaction effect / 交互效应
        interaction_effect = total_effect - allocation_effect - selection_effect
        
        return {
            'Total Effect': total_effect * 252,  # Annualized
            'Allocation Effect': allocation_effect * 252,
            'Selection Effect': selection_effect * 252,
            'Interaction Effect': interaction_effect * 252
        }
    
    def factor_attribution(self, factor_returns=None):
        """Factor-based attribution / 因子归因
        Attributes returns to various risk factors
        将收益归因于各种风险因子
        """
        
        if factor_returns is None:
            # Generate sample factor returns / 生成样本因子收益
            factor_returns = pd.DataFrame({
                'Market': self.benchmark_returns,
                'Size': np.random.randn(len(self.portfolio_returns)) * 0.005,
                'Value': np.random.randn(len(self.portfolio_returns)) * 0.003,
                'Momentum': np.random.randn(len(self.portfolio_returns)) * 0.004,
                'Quality': np.random.randn(len(self.portfolio_returns)) * 0.002
            })
        
        # Run factor regression / 运行因子回归
        from sklearn.linear_model import LinearRegression
        
        model = LinearRegression()
        X = factor_returns.values
        y = self.portfolio_returns.values
        
        # Fit model / 拟合模型
        model.fit(X, y)
        
        # Factor loadings / 因子载荷
        loadings = dict(zip(factor_returns.columns, model.coef_))
        alpha = model.intercept_
        
        # Factor contributions / 因子贡献
        contributions = {}
        for factor, loading in loadings.items():
            factor_contribution = loading * factor_returns[factor].mean() * 252
            contributions[f'{factor} Contribution'] = factor_contribution
        
        contributions['Alpha'] = alpha * 252
        contributions['R-squared'] = model.score(X, y)
        
        return contributions, loadings
    
    def time_attribution(self):
        """Time-based attribution / 基于时间的归因
        Analyzes performance across different time periods
        分析不同时间段的表现
        """
        
        # Monthly returns / 月度收益
        monthly_portfolio = self.portfolio_returns.resample('M').apply(lambda x: (1 + x).prod() - 1)
        monthly_benchmark = self.benchmark_returns.resample('M').apply(lambda x: (1 + x).prod() - 1)
        
        # Quarterly returns / 季度收益
        quarterly_portfolio = self.portfolio_returns.resample('Q').apply(lambda x: (1 + x).prod() - 1)
        quarterly_benchmark = self.benchmark_returns.resample('Q').apply(lambda x: (1 + x).prod() - 1)
        
        return {
            'Monthly': {
                'Portfolio Mean': monthly_portfolio.mean() * 12,
                'Benchmark Mean': monthly_benchmark.mean() * 12,
                'Excess Return': (monthly_portfolio.mean() - monthly_benchmark.mean()) * 12,
                'Information Ratio': (monthly_portfolio - monthly_benchmark).mean() / (monthly_portfolio - monthly_benchmark).std() * np.sqrt(12)
            },
            'Quarterly': {
                'Portfolio Mean': quarterly_portfolio.mean() * 4,
                'Benchmark Mean': quarterly_benchmark.mean() * 4,
                'Excess Return': (quarterly_portfolio.mean() - quarterly_benchmark.mean()) * 4,
                'Information Ratio': (quarterly_portfolio - quarterly_benchmark).mean() / (quarterly_portfolio - quarterly_benchmark).std() * np.sqrt(4)
            }
        }

# Generate sample benchmark returns / 生成样本基准收益
benchmark_returns = returns * 0.8 + np.random.randn(len(returns)) * 0.005

# Perform attribution analysis / 执行归因分析
attribution = PerformanceAttribution(returns, benchmark_returns)

# Brinson attribution / Brinson归因
brinson_results = attribution.brinson_attribution()
print("Brinson Attribution / Brinson归因:")
print("="*50)
for effect, value in brinson_results.items():
    print(f"{effect:20}: {value:>10.2%}")

# Factor attribution / 因子归因
print("\nFactor Attribution / 因子归因:")
print("="*50)
factor_results, loadings = attribution.factor_attribution()
for factor, contribution in factor_results.items():
    if 'R-squared' in factor:
        print(f"{factor:20}: {contribution:>10.4f}")
    else:
        print(f"{factor:20}: {contribution:>10.2%}")

# Time attribution / 时间归因
print("\nTime-based Attribution / 基于时间的归因:")
print("="*50)
time_results = attribution.time_attribution()
for period, metrics in time_results.items():
    print(f"\n{period}:")
    for metric, value in metrics.items():
        if 'Ratio' in metric:
            print(f"  {metric:20}: {value:>10.4f}")
        else:
            print(f"  {metric:20}: {value:>10.2%}")

## 8. Signal Analysis / 信号分析 <a id='signal-analysis'></a>

In [ ]:
# Signal quality analysis / 信号质量分析

def analyze_signal_quality(predictions, returns, forward_periods=[1, 5, 10, 20]):
    """Analyze the quality of trading signals
    分析交易信号的质量
    """
    
    results = {}
    
    for period in forward_periods:
        # Calculate forward returns / 计算前瞻收益
        forward_returns = returns.shift(-period)
        
        # Align predictions and returns / 对齐预测和收益
        aligned = pd.DataFrame({
            'signal': predictions,
            'forward_return': forward_returns
        }).dropna()
        
        if len(aligned) > 0:
            # Information Coefficient (IC) / 信息系数
            ic = aligned['signal'].corr(aligned['forward_return'])
            
            # Rank IC / 排序IC
            from scipy.stats import spearmanr
            rank_ic, _ = spearmanr(aligned['signal'], aligned['forward_return'])
            
            # IC by quantile / 分位数IC
            aligned['signal_quantile'] = pd.qcut(aligned['signal'], q=5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])
            quantile_returns = aligned.groupby('signal_quantile')['forward_return'].mean()
            
            # Win rate by quantile / 分位数胜率
            quantile_winrate = aligned.groupby('signal_quantile')['forward_return'].apply(lambda x: (x > 0).mean())
            
            results[f'{period}D'] = {
                'IC': ic,
                'Rank IC': rank_ic,
                'Top Quintile Return': quantile_returns.iloc[-1] if len(quantile_returns) > 0 else 0,
                'Bottom Quintile Return': quantile_returns.iloc[0] if len(quantile_returns) > 0 else 0,
                'Q5-Q1 Spread': quantile_returns.iloc[-1] - quantile_returns.iloc[0] if len(quantile_returns) > 0 else 0,
                'Top Quintile Win Rate': quantile_winrate.iloc[-1] if len(quantile_winrate) > 0 else 0
            }
    
    return results

# Analyze signal quality / 分析信号质量
signal_quality = analyze_signal_quality(predictions, returns)

print("Signal Quality Analysis / 信号质量分析:")
print("="*60)

# Create results table / 创建结果表格
signal_df = pd.DataFrame(signal_quality).T
print(signal_df.round(4))

# Visualize signal analysis / 可视化信号分析
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. IC over different horizons / 不同时间跨度的IC
periods = list(signal_quality.keys())
ic_values = [signal_quality[p]['IC'] for p in periods]
rank_ic_values = [signal_quality[p]['Rank IC'] for p in periods]

axes[0, 0].bar(range(len(periods)), ic_values, alpha=0.7, label='IC')
axes[0, 0].bar(range(len(periods)), rank_ic_values, alpha=0.7, label='Rank IC')
axes[0, 0].set_xticks(range(len(periods)))
axes[0, 0].set_xticklabels(periods)
axes[0, 0].set_title('Information Coefficient by Horizon / 不同时间跨度的信息系数')
axes[0, 0].set_xlabel('Forward Period / 前瞻期')
axes[0, 0].set_ylabel('IC Value / IC值')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Quintile spreads / 五分位价差
spreads = [signal_quality[p]['Q5-Q1 Spread'] for p in periods]
axes[0, 1].plot(range(len(periods)), spreads, marker='o', linewidth=2, markersize=8)
axes[0, 1].set_xticks(range(len(periods)))
axes[0, 1].set_xticklabels(periods)
axes[0, 1].set_title('Quintile Spread by Horizon / 不同时间跨度的五分位价差')
axes[0, 1].set_xlabel('Forward Period / 前瞻期')
axes[0, 1].set_ylabel('Q5-Q1 Spread / 五分位价差')
axes[0, 1].grid(True, alpha=0.3)

# 3. Signal distribution / 信号分布
axes[1, 0].hist(predictions, bins=50, alpha=0.7, edgecolor='black')
axes[1, 0].set_title('Signal Distribution / 信号分布')
axes[1, 0].set_xlabel('Signal Value / 信号值')
axes[1, 0].set_ylabel('Frequency / 频率')
axes[1, 0].grid(True, alpha=0.3)

# 4. Signal autocorrelation / 信号自相关
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(predictions.dropna(), lags=20, ax=axes[1, 1])
axes[1, 1].set_title('Signal Autocorrelation / 信号自相关')
axes[1, 1].set_xlabel('Lag / 滞后')
axes[1, 1].set_ylabel('Autocorrelation / 自相关')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Transaction Cost Analysis / 交易成本分析 <a id='tca'></a>

In [ ]:
# Transaction cost analysis / 交易成本分析

class TransactionCostAnalysis:
    """Comprehensive transaction cost analysis
    综合交易成本分析
    """
    
    def __init__(self, trades, prices):
        self.trades = trades
        self.prices = prices
    
    def calculate_costs(self, commission_rate=0.001, slippage_model='linear'):
        """Calculate various transaction costs
        计算各种交易成本
        """
        
        total_costs = {
            'Commission': 0,
            'Slippage': 0,
            'Market Impact': 0,
            'Opportunity Cost': 0
        }
        
        # Commission costs / 佣金成本
        trade_value = abs(self.trades).sum()
        total_costs['Commission'] = trade_value * commission_rate
        
        # Slippage costs / 滑点成本
        if slippage_model == 'linear':
            # Linear slippage model / 线性滑点模型
            avg_trade_size = abs(self.trades).mean()
            slippage_rate = 0.0005 * (1 + avg_trade_size / 1000000)
            total_costs['Slippage'] = trade_value * slippage_rate
        elif slippage_model == 'sqrt':
            # Square root model / 平方根模型
            total_costs['Slippage'] = trade_value * np.sqrt(avg_trade_size / 1000000) * 0.001
        
        # Market impact / 市场冲击
        # Simplified Almgren-Chriss model
        volatility = self.prices.pct_change().std()
        avg_volume = 1000000  # Placeholder
        participation_rate = abs(self.trades).sum() / avg_volume
        total_costs['Market Impact'] = trade_value * volatility * np.sqrt(participation_rate) * 0.1
        
        # Opportunity cost / 机会成本
        # Cost of delayed or partial execution
        total_costs['Opportunity Cost'] = trade_value * 0.0002
        
        # Total cost / 总成本
        total_costs['Total'] = sum(total_costs.values())
        
        return total_costs
    
    def analyze_trade_efficiency(self):
        """Analyze trading efficiency metrics
        分析交易效率指标
        """
        
        efficiency_metrics = {}
        
        # Turnover rate / 换手率
        efficiency_metrics['Daily Turnover'] = abs(self.trades).mean()
        efficiency_metrics['Annual Turnover'] = abs(self.trades).mean() * 252
        
        # Trade frequency / 交易频率
        efficiency_metrics['Trade Days'] = (self.trades != 0).sum()
        efficiency_metrics['Trade Frequency'] = (self.trades != 0).mean()
        
        # Average trade size / 平均交易规模
        non_zero_trades = self.trades[self.trades != 0]
        if len(non_zero_trades) > 0:
            efficiency_metrics['Avg Trade Size'] = abs(non_zero_trades).mean()
            efficiency_metrics['Max Trade Size'] = abs(non_zero_trades).max()
            efficiency_metrics['Trade Size Std'] = abs(non_zero_trades).std()
        
        return efficiency_metrics

# Generate sample trade data / 生成样本交易数据
trade_dates = pd.date_range(exp_config['test_start'], exp_config['test_end'], freq='D')
trades = pd.Series(np.random.randn(len(trade_dates)) * 10000, index=trade_dates)
trades[np.random.random(len(trades)) > 0.1] = 0  # Make 90% days no trade

prices = pd.Series(100 + np.random.randn(len(trade_dates)).cumsum(), index=trade_dates)

# Perform TCA / 执行交易成本分析
tca = TransactionCostAnalysis(trades, prices)

# Calculate costs / 计算成本
costs = tca.calculate_costs()
print("Transaction Cost Breakdown / 交易成本分解:")
print("="*50)
for cost_type, amount in costs.items():
    print(f"{cost_type:20}: ${amount:>15,.2f}")

# Analyze efficiency / 分析效率
efficiency = tca.analyze_trade_efficiency()
print("\nTrading Efficiency Metrics / 交易效率指标:")
print("="*50)
for metric, value in efficiency.items():
    if 'Size' in metric:
        print(f"{metric:20}: ${value:>15,.2f}")
    elif 'Frequency' in metric or 'Turnover' in metric:
        print(f"{metric:20}: {value:>15.2%}")
    else:
        print(f"{metric:20}: {value:>15,.0f}")

# Visualize costs / 可视化成本
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Cost breakdown pie chart / 成本分解饼图
cost_values = [costs[k] for k in ['Commission', 'Slippage', 'Market Impact', 'Opportunity Cost']]
cost_labels = ['Commission', 'Slippage', 'Market Impact', 'Opportunity']
axes[0].pie(cost_values, labels=cost_labels, autopct='%1.1f%%', startangle=90)
axes[0].set_title('Transaction Cost Breakdown / 交易成本分解')

# Cumulative costs over time / 累积成本时间序列
cumulative_trades = abs(trades).cumsum()
cumulative_costs = cumulative_trades * 0.002  # Simplified
axes[1].plot(cumulative_costs.index, cumulative_costs.values, linewidth=2)
axes[1].set_title('Cumulative Transaction Costs / 累积交易成本')
axes[1].set_xlabel('Date / 日期')
axes[1].set_ylabel('Cumulative Cost ($) / 累积成本')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Visualization and Reporting / 可视化与报告 <a id='visualization'></a>

In [ ]:
# Comprehensive performance report generation / 综合性能报告生成

def generate_performance_report(portfolio_metrics, risk_metrics, signal_quality, costs):
    """Generate comprehensive performance report
    生成综合性能报告
    """
    
    # Create figure with multiple subplots / 创建多子图
    fig = plt.figure(figsize=(20, 24))
    gs = fig.add_gridspec(6, 3, hspace=0.3, wspace=0.3)
    
    # Title / 标题
    fig.suptitle('Strategy Performance Report / 策略性能报告', fontsize=20, fontweight='bold')
    
    # 1. Cumulative returns comparison / 累积收益对比
    ax1 = fig.add_subplot(gs[0, :])
    ax1.plot(cumulative_returns.index, cumulative_returns.values, label='Strategy', linewidth=2)
    ax1.plot(cumulative_returns.index, (1 + benchmark_returns).cumprod().values, 
            label='Benchmark', linewidth=2, alpha=0.7)
    ax1.set_title('Cumulative Returns / 累积收益', fontsize=14)
    ax1.set_xlabel('Date / 日期')
    ax1.set_ylabel('Cumulative Return / 累积收益')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Monthly returns heatmap / 月度收益热图
    ax2 = fig.add_subplot(gs[1, :])
    monthly_returns = returns.resample('M').apply(lambda x: (1 + x).prod() - 1)
    monthly_pivot = pd.DataFrame({
        'Month': monthly_returns.index.month,
        'Year': monthly_returns.index.year,
        'Return': monthly_returns.values
    }).pivot(index='Month', columns='Year', values='Return')
    
    sns.heatmap(monthly_pivot, annot=True, fmt='.1%', cmap='RdYlGn', center=0, 
               cbar_kws={'label': 'Monthly Return'}, ax=ax2)
    ax2.set_title('Monthly Returns Heatmap / 月度收益热图', fontsize=14)
    
    # 3. Risk metrics / 风险指标
    ax3 = fig.add_subplot(gs[2, 0])
    risk_keys = list(risk_metrics.keys())[:5]
    risk_values = [risk_metrics[k] for k in risk_keys]
    ax3.barh(risk_keys, risk_values)
    ax3.set_title('Risk Metrics / 风险指标', fontsize=12)
    ax3.set_xlabel('Value / 值')
    
    # 4. Signal quality / 信号质量
    ax4 = fig.add_subplot(gs[2, 1])
    signal_df = pd.DataFrame(signal_quality).T
    signal_df[['IC', 'Rank IC']].plot(kind='bar', ax=ax4)
    ax4.set_title('Signal Quality / 信号质量', fontsize=12)
    ax4.set_xlabel('Period / 期间')
    ax4.set_ylabel('IC Value / IC值')
    ax4.legend()
    
    # 5. Cost breakdown / 成本分解
    ax5 = fig.add_subplot(gs[2, 2])
    cost_values = [costs[k] for k in ['Commission', 'Slippage', 'Market Impact', 'Opportunity Cost']]
    cost_labels = ['Commission', 'Slippage', 'Market Impact', 'Opportunity']
    ax5.pie(cost_values, labels=cost_labels, autopct='%1.1f%%', startangle=90)
    ax5.set_title('Transaction Costs / 交易成本', fontsize=12)
    
    # 6. Rolling metrics / 滚动指标
    ax6 = fig.add_subplot(gs[3, :])
    rolling_sharpe = (returns.rolling(60).mean() / returns.rolling(60).std()) * np.sqrt(252)
    ax6.plot(rolling_sharpe.index, rolling_sharpe.values, label='Rolling Sharpe (60D)', linewidth=2)
    ax6.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    ax6.axhline(y=1, color='green', linestyle='--', alpha=0.5)
    ax6.set_title('Rolling Sharpe Ratio / 滚动夏普比率', fontsize=14)
    ax6.set_xlabel('Date / 日期')
    ax6.set_ylabel('Sharpe Ratio / 夏普比率')
    ax6.legend()
    ax6.grid(True, alpha=0.3)
    
    # 7. Drawdown analysis / 回撤分析
    ax7 = fig.add_subplot(gs[4, :])
    running_max = cumulative_returns.expanding().max()
    drawdown = (cumulative_returns - running_max) / running_max
    ax7.fill_between(drawdown.index, drawdown.values, 0, alpha=0.7, color='red')
    ax7.set_title('Drawdown Analysis / 回撤分析', fontsize=14)
    ax7.set_xlabel('Date / 日期')
    ax7.set_ylabel('Drawdown / 回撤')
    ax7.grid(True, alpha=0.3)
    
    # 8. Performance summary table / 性能摘要表
    ax8 = fig.add_subplot(gs[5, :])
    ax8.axis('tight')
    ax8.axis('off')
    
    # Create summary data / 创建摘要数据
    summary_data = [
        ['Metric / 指标', 'Value / 值'],
        ['Total Return / 总收益', f"{(cumulative_returns.iloc[-1] - 1):.2%}"],
        ['Annual Return / 年化收益', f"{metrics['Annual Return']:.2%}"],
        ['Volatility / 波动率', f"{metrics['Volatility']:.2%}"],
        ['Sharpe Ratio / 夏普比率', f"{metrics['Sharpe Ratio']:.4f}"],
        ['Max Drawdown / 最大回撤', f"{metrics['Max Drawdown']:.2%}"],
        ['Calmar Ratio / 卡尔玛比率', f"{metrics['Calmar Ratio']:.4f}"],
        ['Win Rate / 胜率', f"{metrics['Win Rate']:.2%}"],
        ['Best Day / 最佳日', f"{metrics['Best Day']:.2%}"],
        ['Worst Day / 最差日', f"{metrics['Worst Day']:.2%}"],
        ['Total Cost / 总成本', f"${costs['Total']:,.2f}"]
    ]
    
    table = ax8.table(cellText=summary_data[1:], colLabels=summary_data[0],
                     cellLoc='center', loc='center', colWidths=[0.5, 0.5])
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1, 2)
    
    # Style the table / 设置表格样式
    for i in range(len(summary_data)):
        if i == 0:
            for j in range(2):
                table[(i, j)].set_facecolor('#40466e')
                table[(i, j)].set_text_props(weight='bold', color='white')
        else:
            for j in range(2):
                table[(i, j)].set_facecolor('#f0f0f0' if i % 2 == 0 else 'white')
    
    plt.tight_layout()
    return fig

# Generate report / 生成报告
report_fig = generate_performance_report(portfolio_metrics, risk_metrics, signal_quality, costs)
plt.show()

# Save report / 保存报告
report_fig.savefig('performance_report.pdf', dpi=300, bbox_inches='tight')
print("\n✅ Performance report saved as 'performance_report.pdf'")

## Summary / 总结

### What we covered / 本节内容

1. **Backtesting Framework** - Complete infrastructure for strategy testing / 完整的策略测试基础设施
2. **Trading Strategies** - Top-K, signal-based, and custom strategies / Top-K、基于信号和自定义策略
3. **Execution Management** - Order generation and execution with realistic constraints / 带现实约束的订单生成和执行
4. **Portfolio Analysis** - Comprehensive performance metrics and visualization / 综合性能指标和可视化
5. **Risk Metrics** - VaR, CVaR, drawdown, and risk decomposition / VaR、CVaR、回撤和风险分解
6. **Performance Attribution** - Brinson, factor, and time-based attribution / Brinson、因子和基于时间的归因
7. **Signal Analysis** - IC, Rank IC, and signal quality metrics / IC、Rank IC和信号质量指标
8. **Transaction Cost Analysis** - Commission, slippage, and market impact / 佣金、滑点和市场冲击
9. **Reporting** - Professional performance reports and dashboards / 专业的性能报告和仪表板

### Key Takeaways / 关键要点

- **Realistic Backtesting**: Include transaction costs and market constraints / 包含交易成本和市场约束
- **Risk Management**: Monitor multiple risk metrics continuously / 持续监控多个风险指标
- **Signal Quality**: Validate predictions before deployment / 部署前验证预测
- **Attribution Analysis**: Understand sources of returns / 理解收益来源
- **Cost Awareness**: Transaction costs can significantly impact returns / 交易成本可能显著影响收益

### Next Steps / 下一步

Continue with **[05_utils_and_helpers.ipynb](./05_utils_and_helpers.ipynb)** to learn about:
- Utility functions and helpers / 工具函数和辅助功能
- Advanced Qlib features / 高级Qlib功能
- Best practices and tips / 最佳实践和技巧